In [1]:
from andersan_backend import observes
from logging import getLogger, basicConfig, INFO, DEBUG

basicConfig(level=DEBUG)
# 特定の日時の各地理院メッシュ点での測定値をとってくる例
isodate = "2024-09-03T11:00+09:00"
zoom = 12
table = observes("kanagawa", isodate, zoom)
table

DEBUG:root:["kanagawa", "2024-09-03T11:00+09:00", 12]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'observes.sqlite'
DEBUG:sqlitedict:closing SqliteDict(observes.sqlite)
DEBUG:sqlitedict.SqliteMultithread:received: --close--, send: --no more--


,lon,lat,X,Y,Z,NMHC,OX,NOX,TEMP,WX,WY
timestamp,,,,,,,,,,,
2024-09-03 11:00:00+09:00,138.867188,35.675147,3628,1613,12,5.009667,29.662173,4.168914,NaN,3.645199,6.866597
2024-09-03 11:00:00+09:00,138.955078,35.675147,3629,1613,12,6.061962,29.931887,4.034056,NaN,7.417923,8.920602
2024-09-03 11:00:00+09:00,139.042969,35.675147,3630,1613,12,7.114257,29.801940,4.254478,NaN,9.713605,7.276378
2024-09-03 11:00:00+09:00,139.130859,35.675147,3631,1613,12,13.069004,28.809011,6.124183,NaN,2.716571,-11.925141
2024-09-03 11:00:00+09:00,139.218750,35.675147,3632,1613,12,14.828828,30.816232,9.889436,225.103637,-0.923588,-3.181381
...,...,...,...,...,...,...,...,...,...,...,...
2024-09-03 11:00:00+09:00,139.394531,35.245619,3634,1619,12,9.634303,29.094636,13.409776,NaN,10.525069,23.478489
2024-09-03 11:00:00+09:00,139.482422,35.245619,3635,1619,12,9.630303,28.063244,10.740799,248.469333,15.727899,23.622243
2024-09-03 11:00:00+09:00,139.570312,35.245619,3636,1619,12,9.626304,29.742651,5.197268,250.379126,21.481590,22.785760


指定時刻のデータをとってきて地図にする流れ。

In [5]:
from folium import Map, LayerControl  # , Choropleth, FeatureGroup
from andersan_backend import prefecture_ranges
from andersan_map import map_layer
import numpy as np
from andersan import tile

# 日時の指定
isodate = "2024-08-31T00:00+09:00"
zoom = 12
table = observes("kanagawa", isodate, zoom)
# shape = len(table.Y.unique()), len(table.X.unique())
pref_range = prefecture_ranges["kanagawa"]
tiles, shape = tile.tiles(zoom, pref_range)
# mapに重ねるレイヤ
layers = []

# 測定値
for item in ["NMHC", "OX", "NOX", "TEMP", "WX", "WY"]:
    values = table[item].to_numpy()
    layers.append(
        map_layer("kanagawa", values.reshape(shape), item)
    )

# 地図の中心位置
center = np.mean(pref_range, axis=0)[::-1]

# 地図を作る
m = Map(center, zoom_start=10)

# レイヤをオーバーレイする
for layer in layers:
    layer.add_to(m)

# 切り替えボタン(もっと見易くしたい)
LayerControl().add_to(m)

# GroupedLayerControl({"NOX": layers[0]}, ["NOX"]).add_to(m)

# 表示
m

DEBUG:root:["kanagawa", "2024-08-31T00:00+09:00", 12]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'observes.sqlite'
DEBUG:sqlitedict:closing SqliteDict(observes.sqlite)
DEBUG:sqlitedict.SqliteMultithread:received: --close--, send: --no more--


6 項目を空間方向で補間し、欠測格子点がなくなった。これを毎日 24 時間前まで入手する。


次は、各格子点の予測値を OpenMeteo から入手して、同じように表示する。

G0L は OM から何を読んでいるか。(datatype1)

```
            "temperature_2m",
            "cloud_cover",
            "pressure_msl",
            "shortwave_radiation",
```

と`weathercode`

OpenMeteo の API では、指定された緯度経度とはすこしずれた場所の情報が送られてきているように思えるがそんなことはないか?


In [6]:
import numpy as np
from logging import getLogger, basicConfig, INFO, DEBUG
from andersan_backend import openmeteo_tiles, prefecture_ranges, OPENMETEO_ITEMS
import datetime
from folium import Map, LayerControl  # , Choropleth, FeatureGroup
from andersan_map import map_layer
from andersan_backend import prefecture_ranges
basicConfig(level=INFO)

zoom = 12  # 14にしてもOpenMeteoのほうがそこまで解像度がない。
isodate = "2024-08-31T00:00+09:00"
all_forecast_dataframe = openmeteo_tiles("kanagawa", isodate, zoom)
# shape = len(all_forecast_dataframe.Y.unique()), len(all_forecast_dataframe.X.unique())
dt = datetime.datetime.fromisoformat(isodate)
pref_range = prefecture_ranges["kanagawa"]
tiles, shape = tile.tiles(zoom, pref_range)

# レイヤごとに作画する
layers = []
for item in OPENMETEO_ITEMS:
    values = all_forecast_dataframe[all_forecast_dataframe.date == dt][
        [item]
    ].to_numpy()[:, 0]

    # 新しいFoliumレイヤを定義する
    layers.append(
        map_layer("kanagawa", values.reshape(shape), item)
    )

# 地図の中心位置
center = np.mean(pref_range, axis=0)[::-1]

# 地図を作る
m = Map(center, zoom_start=10)

# レイヤをオーバーレイする
for layer in layers:
    layer.add_to(m)

# 切り替えボタン(もっと見易くしたい)
LayerControl().add_to(m)

# 表示
m

DEBUG:root:["kanagawa", "2024-08-31", 12]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'openmeteo.sqlite'
DEBUG:sqlitedict:closing SqliteDict(openmeteo.sqlite)
DEBUG:sqlitedict.SqliteMultithread:received: --close--, send: --no more--
/AIR/andersan-api/andersan_map.py:9: RuntimeWarning: invalid value encountered in divide
  normed_data = (array - np.nanmin(array)) / (np.nanmax(array) - np.nanmin(array))


本番テストに移行する前に: 標準化の方法を統一する。各測定量の標準化方法を固定し、データの多寡によらず同じ標準化が行われるようにする。


In [13]:
import pandas as pd
# import matplotlib.pyplot as plt
# import folium
import numpy as np
# # import requests_cache
# # from retry_requests import retry
from logging import getLogger, basicConfig, INFO, DEBUG
from andersan_backend import observes, openmeteo_tiles, prefecture_ranges, OPENMETEO_ITEMS
import datetime
# from folium import Map, LayerControl  # , Choropleth, FeatureGroup
from datetime import timedelta


import json

def X_instant(pref_name:str, isodate:str, zoom:int, lookback=24, forecast=8, 
    items = ("NMHC", "OX", "NOX", "TEMP", "WX", "WY"),
    noaa_cols = (
        "temperature_2m",
        "cloud_cover",
        "pressure_msl",
        "shortwave_radiation",
        "wind_speed_10m",
    )):

    observes_table = pd.DataFrame()
    timeorigin = datetime.datetime.fromisoformat(isodate)
    for delta in range(-23, 1):
        dt = timeorigin + timedelta(hours=delta)
        table = observes("kanagawa", dt.isoformat(), zoom)
        observes_table = pd.concat([observes_table, table], axis=0)
    # 風向をここで加えておく。

    observes_table

    assert timeorigin.hour < 16, "予測値が翌日にまたがるケースはまだ対応していません。"
    all_forecast_dataframe = openmeteo_tiles("kanagawa", isodate, zoom)
    timebegin = timeorigin + timedelta(hours=1)
    timeend = timeorigin + timedelta(hours=8)
    tiles = np.unique(all_forecast_dataframe[["X", "Y"]].to_numpy(), axis=0)
    # print(tiles)

    X0 = np.zeros([len(tiles), 24, 6])
    X2 = np.zeros([len(tiles), 8, 5])
    X3 = np.zeros([len(tiles), 8], dtype=int)
    for j, (tileX, tileY) in enumerate(tiles):
        for i, item in enumerate(items):
            X0[j, :, i] = observes_table[
                (observes_table.X == tileX) & (observes_table.Y == tileY)
            ][item].to_numpy()
        # print(X0)

        for i, item in enumerate(noaa_cols):
            X2[j, :, i] = all_forecast_dataframe[
                (all_forecast_dataframe.X == tileX)
                & (all_forecast_dataframe.Y == tileY)
                & (all_forecast_dataframe.date >= timebegin)
                & (all_forecast_dataframe.date <= timeend)
            ][item]

        X3[j, :] = all_forecast_dataframe[
            (all_forecast_dataframe.X == tileX)
            & (all_forecast_dataframe.Y == tileY)
            & (all_forecast_dataframe.date >= timebegin)
            & (all_forecast_dataframe.date <= timeend)
        ]["weather_code"]

    X = {
        "Input_lookbacks": X0,
        "Input_forecasts": X2,
        "Input_weathercodes": X3,
    }


    stdfilename = "../andersan-train/datatype3/standards.json"
    logger.info(f"Standardization with {stdfilename}")

    with open(stdfilename) as f:
        specs = json.load(f)

    for label in specs:
        for icol in range(X[label].shape[-1]):
            average = specs[label][icol]["average"]
            std = specs[label][icol]["std"]
            X[label][:, :, icol] = (X[label][:, :, icol] - average) / std

    return X

basicConfig(level=DEBUG)
# 日時の指定
isodate = "2024-09-03T06:00+09:00"
zoom = 12

logger = getLogger()

X = X_instant("kanagawa", isodate, zoom)
X

DEBUG:root:["kanagawa", "2024-09-02T07:00:00+09:00", 12]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'observes.sqlite'
DEBUG:sqlitedict:closing SqliteDict(observes.sqlite)
DEBUG:sqlitedict.SqliteMultithread:received: --close--, send: --no more--
DEBUG:root:["kanagawa", "2024-09-02T08:00:00+09:00", 12]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'observes.sqlite'
DEBUG:sqlitedict:closing SqliteDict(observes.sqlite)
DEBUG:sqlitedict.SqliteMultithread:received: --close--, send: --no more--
DEBUG:root:["kanagawa", "2024-09-02T09:00:00+09:00", 12]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'observes.sqlite'
DEBUG:sqlitedict:closing SqliteDict(observes.sqlite)
DEBUG:sqlitedict.SqliteMultithread:received: --close--, send: --no more--
DEBUG:root:["kanagawa", "2024-09-02T10:00:00+09:00", 12]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'observes.sqlite'
DEBUG:sqlitedict:closing SqliteDict(observes.sqlite)
DEBUG:sqlitedict.SqliteMultithread:received: --close--, send: 

{'Input_lookbacks': array([[[-1.41713321e+00, -1.07762183e+00, -7.77422300e-01,
                      nan, -9.26209540e-02, -4.59112231e-01],
         [-1.42468538e+00, -8.54761771e-01, -9.93970175e-01,
                      nan,  4.89266875e-02, -2.84087438e-01],
         [            nan, -5.17667287e-01, -9.21625013e-01,
                      nan,  3.33563125e-01, -1.05304572e-01],
         ...,
         [-1.25368081e+00, -1.22350239e+00, -1.00388519e+00,
                      nan, -1.06411494e-03,  1.55176966e-01],
         [-1.39621580e+00, -1.19841198e+00, -1.03832400e+00,
                      nan, -4.16731825e-01,  4.58099210e-01],
         [-1.26123298e+00, -1.19280318e+00, -8.11861110e-01,
                      nan, -1.25150024e-01,  1.47748002e-01]],
 
        [[-1.44123777e+00, -1.02539784e+00, -3.26280183e-01,
                      nan,  1.65853581e-01, -1.88741405e-01],
         [-1.46583931e+00, -7.96068228e-01, -7.73563152e-01,
                      nan,  5.66918247e-01

In [14]:
# 実測の未来値
forecasts_table = pd.DataFrame()
timeorigin = datetime.datetime.fromisoformat(isodate)
for delta in range(1, 9):
    dt = timeorigin + timedelta(hours=delta)
    table = observes("kanagawa", dt.isoformat(), zoom)
    forecasts_table = pd.concat([forecasts_table, table], axis=0)

forecasts_table
# 14 sec

DEBUG:root:["kanagawa", "2024-09-03T07:00:00+09:00", 12]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'observes.sqlite'
DEBUG:sqlitedict:closing SqliteDict(observes.sqlite)
DEBUG:sqlitedict.SqliteMultithread:received: --close--, send: --no more--
DEBUG:root:["kanagawa", "2024-09-03T08:00:00+09:00", 12]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'observes.sqlite'
DEBUG:sqlitedict:closing SqliteDict(observes.sqlite)
DEBUG:sqlitedict.SqliteMultithread:received: --close--, send: --no more--
DEBUG:root:["kanagawa", "2024-09-03T09:00:00+09:00", 12]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'observes.sqlite'
DEBUG:sqlitedict:closing SqliteDict(observes.sqlite)
DEBUG:sqlitedict.SqliteMultithread:received: --close--, send: --no more--
DEBUG:root:["kanagawa", "2024-09-03T10:00:00+09:00", 12]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'observes.sqlite'
DEBUG:sqlitedict:closing SqliteDict(observes.sqlite)
DEBUG:sqlitedict.SqliteMultithread:received: --close--, send: 

,lon,lat,X,Y,Z,NMHC,OX,NOX,TEMP,WX,WY
timestamp,,,,,,,,,,,
2024-09-03 07:00:00+09:00,138.867188,35.675147,3628,1613,12,7.513886,4.391728,3.982308,NaN,-2.130312,2.781426
2024-09-03 07:00:00+09:00,138.955078,35.675147,3629,1613,12,8.119809,2.871266,3.657876,NaN,-0.900195,1.812375
2024-09-03 07:00:00+09:00,139.042969,35.675147,3630,1613,12,8.725733,1.528209,4.037165,NaN,0.210290,1.035895
2024-09-03 07:00:00+09:00,139.130859,35.675147,3631,1613,12,11.687928,2.214204,7.634420,NaN,1.002197,2.818942
2024-09-03 07:00:00+09:00,139.218750,35.675147,3632,1613,12,11.694898,5.714012,5.986039,240.930452,3.551471,8.777350
...,...,...,...,...,...,...,...,...,...,...,...
2024-09-03 14:00:00+09:00,139.394531,35.245619,3634,1619,12,10.777813,34.751570,9.079412,NaN,14.129464,9.872042
2024-09-03 14:00:00+09:00,139.482422,35.245619,3635,1619,12,10.168185,34.690763,8.452445,264.179687,16.467456,14.536178
2024-09-03 14:00:00+09:00,139.570312,35.245619,3636,1619,12,9.511928,33.603663,10.424960,259.862640,16.994094,20.174095


In [15]:
from andersan import tile

# そのうちOXのみ
# 地理院メッシュの間隔
pref_range = np.array(prefecture_ranges["kanagawa"])  # lon,lat
tiles, shape = tile.tiles(zoom, pref_range)

y = np.zeros([len(tiles), 8])
for j, (tileX, tileY) in enumerate(tiles):
    y[j, :] = forecasts_table[
        (forecasts_table.X == tileX) & (forecasts_table.Y == tileY)
    ].OX.to_numpy()

y
# 0.1 sec

array([[ 4.39172753,  4.8664708 , 10.88416303, 22.57792169, 29.66217266,
        32.48904878, 36.52443325, 37.42249005],
       [ 2.87126639,  5.65019086, 11.99231446, 22.35905488, 29.93188732,
        31.76004542, 36.44429262, 36.52933658],
       [ 1.52820884,  6.22567072, 13.27664052, 22.19055924, 29.80193984,
        30.93522492, 36.18220184, 35.75302308],
       [ 2.21420426,  6.89734151, 15.56826307, 22.34392769, 28.80901068,
        29.10335335, 34.12431045, 34.99389215],
       [ 5.71401203, 12.31272549, 17.04774041, 22.44914872, 30.81623222,
        26.56469084, 30.45619028, 32.04774041],
       [11.1664528 , 15.79317149, 19.12858254, 23.69606894, 28.07771908,
        26.02524684, 30.54847273, 34.98098633],
       [18.52174391, 19.95054012, 27.36570879, 29.96248424, 30.20966207,
        28.47825609, 32.07503153, 37.51577185],
       [16.59024458, 19.92402647, 29.62890418, 31.54439129, 32.15183083,
        30.98280504, 31.94841512, 36.56877994],
       [16.58204213, 24.98342672

In [16]:
import keras

basename = "andersan0_1.py"
model = keras.models.load_model(f"../andersan-train/{basename}.best.keras")
# model.summary()

pred = model.predict(X)
pred**0.5
# 6.7 sec

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 142ms/step


array([[       nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        n

In [17]:
import folium
from folium import Map, LayerControl, Choropleth, FeatureGroup
import matplotlib.pyplot as plt
from andersan_map import map_layer

def colorize(array, cmap="viridis"):
    # 値を色に変換
    normed_data = array / 120
    cm = plt.cm.get_cmap(cmap)
    return cm(normed_data)


layers = []
foresee = 8
pref_range = prefecture_ranges["kanagawa"]

maps = {"observes": y[:, foresee - 1], "predicts": (pred**0.5)[:, foresee - 1]}
for name, values in maps.items():
    # 新しいFoliumレイヤを定義する
    layers.append(
        map_layer("kanagawa", values.reshape(shape), f"{name} +{foresee} hrs")
    )

# 地図の中心位置
center = np.mean(pref_range, axis=0)[::-1]

# 地図を作る
m = Map(center, zoom_start=10)

# レイヤをオーバーレイする
for layer in layers:
    layer.add_to(m)

# 切り替えボタン(もっと見易くしたい)
LayerControl().add_to(m)

# 表示
m

だいぶ簡潔にできた。